# vLLM Quickstart

## Offline Batched Inference

In [ ]:
!pip install vllm torch  triton

In [ ]:
from vllm import LLM, SamplingParams

# Load a pre-trained Hugging Face model
model_name = "gpt2"  # Replace with your desired model

# Create an LLM.
#gpu_memory_utilization: The ratio (between 0 and 1) of GPU memory to
#reserve for the model weights, activations, and KV cache. Higher
#values will increase the KV cache size and thus improve the model's
#throughput. However, if the value is too high, it may cause out-of-
# memory (OOM) errors.
llm = LLM(model_name, gpu_memory_utilization = 0.5)

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


Restart Colab to free GPU memory and rerun the demo above.

In [ ]:
import os
#os._exit(00)

In [ ]:
!nvidia-smi

Sun Dec  1 04:51:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              26W /  70W |   7525MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--